In [1]:
import torch
import numpy as np
import glob
import argparse

In [2]:
! pwd

/ssl-jet-vol-v2/JetCLR/notebooks


In [3]:
parser = argparse.ArgumentParser()
args = parser.parse_args(args=[])
args.load_labels = True
args.raw = 1
args.percent = 5

## Import data using JetClassDataset

In [9]:
import sys
sys.path.insert(0, '../src')
from modules.dataset.dataset import JetClassDataset
from torch.utils.data import DataLoader

# Initialize your custom dataset
dataset_path = '/ssl-jet-vol-v2/JetClass/processed/raw'
train_dataset = JetClassDataset(
        dataset_path,
        flag="train",
        args=args,
        logfile=sys.stdout,
        load_labels=False,
    )

Number of data files: 50
Data files: ['data_0.pt', 'data_1.pt', 'data_10.pt', 'data_11.pt', 'data_12.pt', 'data_13.pt', 'data_14.pt', 'data_15.pt', 'data_16.pt', 'data_17.pt', 'data_18.pt', 'data_19.pt', 'data_2.pt', 'data_20.pt', 'data_21.pt', 'data_22.pt', 'data_23.pt', 'data_24.pt', 'data_25.pt', 'data_26.pt', 'data_27.pt', 'data_28.pt', 'data_29.pt', 'data_3.pt', 'data_30.pt', 'data_31.pt', 'data_32.pt', 'data_33.pt', 'data_34.pt', 'data_35.pt', 'data_36.pt', 'data_37.pt', 'data_38.pt', 'data_39.pt', 'data_4.pt', 'data_40.pt', 'data_41.pt', 'data_42.pt', 'data_43.pt', 'data_44.pt', 'data_45.pt', 'data_46.pt', 'data_47.pt', 'data_48.pt', 'data_49.pt', 'data_5.pt', 'data_6.pt', 'data_7.pt', 'data_8.pt', 'data_9.pt']
jets per file: 100000
total jets: 5000000


In [15]:
from tqdm import tqdm
# Create a DataLoader
data_loader = DataLoader(train_dataset, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

batch_counter = 0
for data in tqdm(data_loader):
    pass
#     print(batch_counter)
#     batch_counter += 1
    # Now `data` contains your batch of jets, and `labels` contains the corresponding labels
#     print(data.shape)  # This will show the shape of your batch of jets
#     print(labels.shape)  # This will show the shape of your batch of labels
#     break

  4%|██▏                                                    | 1562/39063 [00:06<02:33, 244.83it/s]


KeyboardInterrupt: 

In [11]:
input_dim = train_dataset.get_sample_shape()[0]

In [12]:
input_dim

7

## Import data using simclr functions

In [ ]:
# load data
def load_data(dataset_path, flag, n_files=-1):
    if args.raw:
        data_files = glob.glob(f"{dataset_path}/raw/raw_{flag}_{args.percent}%/data/*")
    else:
        data_files = glob.glob(f"{dataset_path}/{flag}_{args.percent}%/data/*")

    data = []
    for i, file in enumerate(data_files):
        data.append(torch.load(file)) 
        print(f"--- loaded file {file} from `{flag}_{args.percent}` directory")
        if n_files != -1 and i == n_files - 1:
            break

    return data


def load_labels(dataset_path, flag, n_files=-1):
    if args.raw:
        data_files = glob.glob(f"{dataset_path}/raw/raw_{flag}_{args.percent}%/label/*")
    else:
        data_files = glob.glob(f"{dataset_path}/{flag}_{args.percent}%/label/*")

    data = []
    for i, file in enumerate(data_files):
        data.append(torch.load(file)) 
        print(f"--- loaded file {file} from `{flag}_{args.percent}` directory")
        if n_files != -1 and i == n_files - 1:
            break

    return data

In [ ]:
args.percent = 5
print( "loading data")
data = load_data("/ssl-jet-vol-v2/JetClass/processed", "val")
labels = load_labels("/ssl-jet-vol-v2/JetClass/processed", "val")

In [ ]:
for file in data:
    print(file.shape)

In [ ]:
for file in labels:
    print(file.shape)

In [ ]:
torch.save(data[2][:50000], "data_0.pt")
torch.save(data[2][50000:], "data_1.pt")
torch.save(data[0][:50000], "data_2.pt")
torch.save(data[0][50000:], "data_3.pt")
torch.save(data[1], "data_4.pt")

In [ ]:
torch.save(labels[1][:50000], "labels_0.pt")
torch.save(labels[1][50000:], "labels_0_1.pt")
torch.save(labels[2][:50000], "labels_0_2.pt")
torch.save(labels[2][50000:], "labels_0_3.pt")
torch.save(labels[0], "labels_0_4.pt")

In [ ]:
hbb_raw = torch.load("/ssl-jet-vol-v2/JetClass/processed/raw/val/data/HToBB_120.pt")
hbb = torch.load("/ssl-jet-vol-v2/JetClass/processed/val/data/HToBB_120.pt")
labels_hbb_raw = torch.load("/ssl-jet-vol-v2/JetClass/processed/raw/val/label/labels_HToBB_120.pt")
hbb.shape

In [ ]:
hbb_raw.dtype

In [ ]:
labels_hbb_raw.dtype

In [ ]:
hbb_raw[:10, 2:, :20] - hbb[:10, 2:, :20]

In [ ]:
hbb[:10, 0, :20]

In [ ]:
data[0].shape

In [ ]:
tr_dat_in = torch.stack(data, axis=0)  # Concatenate along the first axis
tr_lab_in = torch.stack(labels, axis=0)

# input dim to the transformer -> (pt,eta,phi)
input_dim = tr_dat_in.shape
print(input_dim)